In [14]:
from mako.template import Template
import pandas as pd


In [8]:
df_article = pd.read_pickle('./data/article_full.pkl')
df_comment = pd.read_pickle('./data/comment_full.pkl')
genHTML('171980706',df_article,df_comment)

In [7]:
hashlib.md5('hello'.encode()).hexdigest()

'5d41402abc4b2a76b9719d911017c592'

In [25]:
import pandas as pd
import hashlib
import numpy as np

df_comment = pd.read_pickle('../data/comment_full.pkl')
def comment2tag(df_comment):
    df_comment['md5'] = df_comment['comment'].apply(lambda x: hashlib.md5((x.replace('<strike>','').replace('</strike>','')).encode()).hexdigest())
    return df_comment

def table2tag(df_comment):
    df_comment['md5'] = df_comment['comment'].apply(lambda x: hashlib.md5((x.replace('<strike>','').replace('</strike>','')).encode()).hexdigest())
    df = df_comment.sort_values('comment_date',ascending=True)[['comment','reply','md5']].drop_duplicates()
    df = df_comment.sort_values('comment_date',ascending=True)[['comment','reply','md5']].drop_duplicates()
    df_table = pd.DataFrame()
    nrow = 400
    for i in range(round(np.ceil(len(df)/nrow))):
        for j in range(nrow):
            idx = nrow * i + j
            if idx < len(df):
                comment = df['comment'].iloc[idx].replace('<strike>','').replace('</strike>','').replace(' ','').replace('　','').replace('=','').replace('&gt;','').replace('&ldquo;','').replace('&rdquo;','')
                df_table.loc[j, f'code_{i}'] = comment[:1] + df['md5'].iloc[idx]
                if len(df['reply'].iloc[idx]) > 300:
                    df_table.loc[j, f'tag_{i}'] = '问答1000/'
                else:
                    df_table.loc[j, f'tag_{i}'] = 'empty/'
    return df_table

table2tag(df_comment).to_excel('table.xlsx')

In [26]:
df_comment = pd.read_pickle('../data/comment_full.pkl')
df_table = pd.read_excel('table.xlsx',index_col=0)

def table2tag(df_comment, df_table):
    df_comment['md5'] = df_comment['comment'].apply(lambda x: hashlib.md5((x.replace('<strike>','').replace('</strike>','')).encode()).hexdigest())
    df_tag = pd.DataFrame()
    for i in range(round(len(df_table.columns)/2)):
        df_tag = pd.concat([df_tag, df_table[[f'code_{i}',f'tag_{i}']].rename(columns={f'code_{i}':'code',f'tag_{i}':'tag'})],axis=0)
    df_tag = df_tag.loc[~df_tag.code.isna()].reset_index(drop=True)
    df_tag['md5'] = df_tag['code'].apply(lambda x: x[-32:])
    df_comment_tag = pd.merge(df_comment, df_tag, on='md5',how='left')
    return df_comment_tag

In [113]:
df_comment.loc[df_comment.md5 == 'd0736836f77e093866f11d825769ea1a']

,comment,reply,nickname,comment_date,first_reply_date,latest_reply_date,id,md5
9360,I am happy to recommend some websites. <br/><a...,Thank you very much.,caspase,2015-05-21 00:00:00,NaT,NaT,108908735,d0736836f77e093866f11d825769ea1a


In [19]:
df_comment_tag

,comment,reply,nickname,comment_date,first_reply_date,latest_reply_date,id,md5,code,tag
0,<strike>關於台灣大選大陸和台灣統派有一種看法關於台灣大選大陸和台灣統派有一種看法認為...,,混沌,2023-05-11 18:02:00,NaT,NaT,179164306,f3b7c0f539a73a7a5ca6a355aef5b729,關f3b7c0f539a73a7a5ca6a355aef5b729,问答1000/
1,希望王老师的主张能被采纳。不过我觉得尹锡悦政权支持率一直走低，不满的情绪仍在发酵。如果爆出一...,<br>這是精確邏輯必須面對的阻力：超前浮面態勢太遠，一般人做分析不是依賴邏輯，而是凴經驗、...,韭菜盒子,2023-05-11 13:39:00,2023-05-12 03:31:00,2023-05-12 03:31:00,179164306,34866fc4c6b9b9e6704bae18e5a459dd,希34866fc4c6b9b9e6704bae18e5a459dd,empty/
2,"<strike>懂下圍棋的都知道:氣長者勝。而“氣長""正是所謂民主政體所缺乏的。只要不被挑鬥...",,狐禪,2023-05-08 12:51:00,NaT,NaT,179164306,7eee826bfc08d30ae10e5ff8254856dd,懂7eee826bfc08d30ae10e5ff8254856dd,empty/
3,后注二提醒我面对统计结果要更加小心谨慎。如果赛制从BO3改成打满3局，那么第二三节比赛也将恢...,<br>不是博弈問題，而只是簡單的概率問題。這個案例其實印證了Occam's Razor的重...,Taizi Huang,2023-05-08 11:31:00,2023-05-11 08:14:00,2023-05-11 08:14:00,129017969,9746db70c1d76301a9a883bed84c5a94,后9746db70c1d76301a9a883bed84c5a94,empty/
4,<strike>笔误马克龙2017年当选总统 </strike>,,龙发,2023-05-08 01:23:00,NaT,NaT,179164306,53ef896d4519f05cdcd1098fd4ba4e86,笔53ef896d4519f05cdcd1098fd4ba4e86,empty/
...,...,...,...,...,...,...,...,...,...,...
9763,向王先生致敬，祝万事如意，我也是观察者的粉，我的看法是&hellip;&hellip;观察者...,对转基因，我的意见在《漫谈食品安全》已经写过了，就是目前所有的转基因都是安全的，不过未来的很...,novalin,2015-02-09 00:00:00,NaT,NaT,108908699,618ba49520664fff87dd768aead86f8b,向618ba49520664fff87dd768aead86f8b,empty/
9764,"我上次发表迴响,说""沱江号是航母杀手""国防部长是胡说八道, 结果被臺湾军方的""婉君""围剿, ...",我用的输入法是美国的Google Pinyin，所以简体字比较方便，我最后再让Google ...,luliu,2015-02-08 00:00:00,NaT,NaT,108908700,497fa4bd4e7032456c3d636465142878,我497fa4bd4e7032456c3d636465142878,empty/
9765,看您的用语和两篇同大陆网友的交流，感觉到您也一定是经常在大陆的论坛上逛，欢迎您多来留言！<b...,观察者的读者意见比较单一。我喜欢到超大军事网去；虽然喷子很多，但是懂事的人也不少，你来我往的...,世界对白,2015-02-07 00:00:00,NaT,NaT,108908699,49cfeab31e44a090b7aaaa9360f04218,看49cfeab31e44a090b7aaaa9360f04218,empty/
9766,您好，有幸拜读您的大作受益匪浅！我也是观察者的粉，未经您允许擅自转载了您的文章，望见谅！&m...,,世界对白,2015-02-07 00:00:00,NaT,NaT,108908699,1a78e12d48f8def38cf4e71444e4eebd,您1a78e12d48f8def38cf4e71444e4eebd,empty/


In [31]:
def genHTML(art_id,df_article,df_comment_tag):
    
    i = df_article.loc[df_article.id == art_id].index
    title = df_article.title[i[0]]
    art_date = df_article.art_date[i[0]]
    post = df_article.post[i[0]]
    
    reply_li = []
    df_comment_id = df_comment_tag.loc[df_comment_tag.id == art_id].sort_values(by=['comment_date','nickname','comment']).drop_duplicates(subset=['comment'],keep='first')
    for j in df_comment_id.index:
        comment = df_comment_id.comment[j]
        if comment:
            reply = re.sub('<br>$','',re.sub('^<br>','',df_comment_id.reply[j])).replace('<br>','<br><br>')

            if reply:
                first_reply_date = df_comment_id.loc[j, 'first_reply_date']
                latest_reply_date = df_comment_id.loc[j, 'latest_reply_date']

                if str(first_reply_date) != 'NaT':
                    if first_reply_date == latest_reply_date:
                        reply = reply + f'<br><div class="TIME">{latest_reply_date.strftime("%Y-%m-%d %H:%M")} 回复</div>'
                    else:
                        reply = reply + f'<br><div class="TIME">{latest_reply_date.strftime("%Y-%m-%d %H:%M")} 修改</div>'

            nickname = df_comment_id.nickname[j]
            comment_date = df_comment_id.comment_date[j].strftime('%Y-%m-%d %H:%M')
            uuid = df_comment_id.md5[j]
            tag_list = ' '.join([f'#{t}' for t in  df_comment_id.tag[j].split('/')])
            striked = '<strike>' in comment
            reply_li.append((uuid,comment,reply,nickname,comment_date,tag_list,striked))
            
    HTML = Template("""<!DOCTYPE html><html><head>    
    <meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name=viewport><meta charset=utf-8>
    <script src="./blog.js"></script>
    <link rel="stylesheet" href="./init.css">
    <title>${title}</title>
    </head>
    <body><div class="BODY">
    <div class="BACK"><a href="../index.html">返回索引页</a></div>
    <h1>${title}</h1>
    <p class="DATE">${date}</p>
    <h4>原文网址：<a href="https://blog.udn.com/MengyuanWang/${art_id}">https://blog.udn.com/MengyuanWang/${art_id}</a></h4>
    <br><br>
    <div class="POST">${post}</div>
    <div class="REPLY_LI">
    <h2>${len(reply_li)} 条留言</h2>
    %for uuid, say, reply, user, time, tag_list, striked in reply_li:
    <div class="LI">
    <div class="USER"><span class="NAME" id=${uuid}>${user}</span>
    <span class="TAG"><label id=${uuid} onclick="delayClick(this.id)">${tag_list}</label></span>
    <div class="TIME">${time}</div></div>
    %if striked:
    <input type="checkbox" class="exp" id="${uuid}_1">
    <div class="text1"><label class="btn" for="${uuid}_1"></label>
    <div class="SAY">${say}</div>
    </div>
    %else:
    <div class="SAY">${say}</div>
    %endif
    %if reply:
    <div class="REPLY">${reply}</div>
    %endif
    </div>
    %endfor
    </div>
    </div>
    <div class="BACK"><a href="../index.html">返回索引页</a></div>
    </body></html>""")
        
    with open('../html/'+art_id+'.html','w',encoding='utf8') as f:
        f.write(HTML.render(title=title,date=art_date,art_id=art_id,post=post,reply_li=reply_li))
    return

In [32]:
import re
df_article = pd.read_pickle('../data/article_full.pkl')
genHTML('108908703',df_article,df_comment_tag)

In [61]:

df_table = pd.DataFrame()
nrow = 400
for i in range(round(np.ceil(len(df)/nrow))):
    for j in range(nrow):
        idx = nrow * i + j
        if idx < len(df):
            first = df['comment'].iloc[idx].replace('<strike>','').replace('</strike>','').replace(' ','').replace('　','').replace('=','').replace('&gt;','').replace('&ldquo;','').replace('&rdquo;','')[:1]
            df_table.loc[j, f'code_{i}'] = first + df['md5'].iloc[idx]
            df_table.loc[j, f'tag_{i}'] = 'null'
df_table.to_excel('table.xlsx')
df_table

,code_0,tag_0,code_1,tag_1,code_2,tag_2,code_3,tag_3,code_4,tag_4,...,code_20,tag_20,code_21,tag_21,code_22,tag_22,code_23,tag_23,code_24,tag_24
0,您1a78e12d48f8def38cf4e71444e4eebd,null,Id0736836f77e093866f11d825769ea1a,null,先d9bb5b7d2e2b9aed1d10a5816f67fcf8,null,T76b3fb86a4ef9ef6241339fb427ca115,null,@67afb8b42b96a538f54bf52a727c0494,null,...,總56eb3494a4ad6baa04c94f0e3e0e06e4,null,Babca5cddef63dbea00384c783da38443,null,請a4d8a1d72be9f076ea261c6466ae8046,null,请b3d300be364904293ca8c72eeb2c8121,null,有01c8f143d388593723dcef5605b47777,null
1,看49cfeab31e44a090b7aaaa9360f04218,null,之cccf8e53081015ea7ee755ac8549f2fa,null,很8483f5db80ad5e96960a2ca5e6cdff95,null,中2a85276fc6fcd1f23202b885558376ea,null,Uef0573542c1b6029eccf3b223e2fce60,null,...,或2d76e6208e9e740323fac54d4e613b22,null,日5a08e1bf1980d3a644ec31167948f96e,null,中a5a0c2979ff4aec0fea0d03cbeb5cb6b,null,回5f18925cafbffa8688bc450e11936236,null,富fc3b3afa0c60cdba6fb7bba4ab273a84,null
2,我497fa4bd4e7032456c3d636465142878,null,身eeba3905186cd0550038e6318668bcc1,null,s2ed4ded0019fd8cfac7c8798875dd663,null,如ae79f281138e57b3dc7a8723f0d99a98,null,看c876509c3d24f4f04a485510d03f01cd,null,...,请9d927770c0b0fd7e229ed84e0aef5bd3,null,很aa79abe0450a585de39e52a0ca5f94ae,null,我85070617b8988a549a6cbf0174275862,null,回d50ee22fa532cd0caa01dee66dd59e80,null,这7352d5b67adbf5ba118e3e526435133b,null
3,向618ba49520664fff87dd768aead86f8b,null,273de97d991210ac7b858ff5a416bcfcb,null,我67f8d4112606b2c42fa6332d6beab8c2,null,@209b835a5945ffe6cb7b9cde3af6bfbb,null,抱944b86ffb97a957dee33e0b3d9efeb53,null,...,回7d348ad209bac426d6176577e38ed5a3,null,翟4d6bd2bf4882d523b11a1177ec90c97b,null,只e54e461fc4b0525a6bd7727387bf0813,null,最06f879f64c0528d425359f57efd79ffd,null,半d597e90e9f4c6954b3e57b84f9c983c0,null
4,茫6e64bfae6f9d2c61a76c1cd8dbebcba5,null,说287e6a4b45afc2fe48719a2cdef17060,null,实a54e51c7934071e51cee046ad06eb234,null,前c585ce024a2d42d97d1b51c594c484d2,null,天ad4569513b1461d8d487878f6b7f3ed0,null,...,“e500111fb2b05306df053e2f5a0e0b1e,null,川522368d60228675c30c6095ec4302470,null,2f56c994a4002becbcda791f730ea997b,null,请7f86657c2a291e5f3e7cc88767b256ee,null,半5c5cf8e976a586c6678f2e3b1b5b9920,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,「1fe7f5f99c01372186062524e9170da9,null,説e44ab6aa2a1d6d8cc56ae1d0b4625a68,null,上7f935abc246aa9a7c2d16b823013cd8c,null,很8a0dc529bc5f9d8775aeab043e49f5b5,null,S2ec58b612374a36da3577f13b483b8f9,null,...,先972b247cfdfd1e1e91e96f25cdcea4fa,null,沙ede6be84ad868576b8812502c9533372,null,国15bdd66635cdef02e219ed6ae2258a55,null,俄986422817db64f68751fc36f42b46297,null,NaN,NaN
396,859b41528a73a5ceba4821d38bbdc3c5a,null,流6d64effaad60b5ccc1617adea22d6d63,null,哎ae612632b648b581e8f0bd2fea74180c,null,我358a146390638fadec81305f2c389e6c,null,给2343403fde6794a23698c296663ca57f,null,...,小677030d5de03aae3548c5f1beba9d6b0,null,最4815d36b643b8a65336e99917f82c890,null,13043955309771becbf531c7ff5393d8d,null,俄8ac4cf1ee60ecc6a98438e01e2f5e3a9,null,NaN,NaN
397,关4fffc1272d393413eb6c25570d576173,null,拜685bffcd1deec5b2d04c27e11267ca54,null,T90ea3b07f1c32f1b70a14c69cc1f971b,null,疾3bca801b7d499b78ad876c00b060c591,null,明63d5c0e6cbbaee86efccea92ac0d0952,null,...,王47b58e6acf52b52924b7e88c20ea7349,null,王ffb4f6e322fa306a409a353b1fa75bd4,null,土337cf6c7e9c6de6e5d09a63df7417b5a,null,今6af27dfcd81b4e7fdcd0891f83683373,null,NaN,NaN
398,我75ef605cf1d199f29868bba1653367c8,null,先0377a5f7f31ca7ccb212ef10f4baa23a,null,我0dbed0e5c9d8cbc0134e6043b5676d66,null,个ca4b901174896b8045e5fc046d76569a,null,先68b269bbab632976723e01404961f031,null,...,先987a7c322a021cd7bc5e35c9499deca4,null,俄2aed6e5039d519ec35a49be6d07c8f05,null,今4d8cc8ba40dcb7412cf695a14da64214,null,有20acf05a5a7d94ffae40e21468b3105c,null,NaN,NaN


In [34]:
df_comment.loc[df_comment.md5 == '98569af0589cf24f51bf87c31e38ce2b']

,comment,reply,nickname,comment_date,first_reply_date,latest_reply_date,id,md5
7390,习近平先自断后路，然后才有王岐山和徐爱生。<br/><br/>这个逻辑很清楚，如果只是为了抓...,2013年习近平刚开始反腐，西方媒体都说这是做做样子，一年后还在搞，他们就改口说是政治斗争，...,caira,2015-10-28 00:00:00,NaT,NaT,108908785,98569af0589cf24f51bf87c31e38ce2b


In [37]:
df = pd.DataFrame(data=[[1,2,3],[4,5,6]],columns=['a','b','c'])
df

,a,b,c
0,1,2,3
1,4,5,6


In [44]:
df1 = pd.DataFrame(data=[[3,10]],columns=['c','d'],index=[4])
df_merge = pd.merge(df,df1,on='c',how='left')
df_merge.loc[df_merge.d.isna(),'d'] = 'empty/'
df_merge

,a,b,c,d
0,1,2,3,10.0
1,4,5,6,empty/
